# Init + Auth

In [1]:
import requests
from pprint import pprint

In [2]:
AUTH_URL      = 'https://accounts.spotify.com/api/token'
BASE_URL      = 'https://api.spotify.com/v1/'

def get_spotify_access_token( authURL = AUTH_URL ):
    """ Get token from Spotify to allow API calls """
    # Original Author: https://stmorse.github.io/journal/spotify-api.html
    CLIENT_ID     = ""
    CLIENT_SECRET = ""

    with open( "../keys/spot_ID.txt" , 'r' ) as f:
        CLIENT_ID = f.readlines()[0].strip()
    
    with open( "../keys/spot_SECRET.txt" , 'r' ) as f:
        CLIENT_SECRET = f.readlines()[0].strip()
    
    if len( CLIENT_ID ) and len( CLIENT_SECRET ):
        print( "Authorization strings have been obtained!" )
    else:
        print( "Auth info is MISSING!" )

    # POST
    auth_response = requests.post( authURL, {
        'grant_type': 'client_credentials',
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
    })
    auth_response_data = auth_response.json()
    access_token       = auth_response_data['access_token']
    return access_token

def get_headers( access_token ):
    """ Get header struct for Spotify API calls """
    return { 'Authorization': 'Bearer {token}'.format( token = access_token ) }

In [3]:
access_token = get_spotify_access_token()

Authorization strings have been obtained!


# Get Playlist Items

In [4]:
playlist_id  = '6KI7A4MWrSM7EyKRUjxIi1'
response = requests.get( BASE_URL + 'playlists/' + playlist_id + '/tracks/', headers = get_headers( access_token ) )
response = response.json()

In [5]:
for key, val in response.items():
    print( key, ':', type( val ) )

href : <class 'str'>
items : <class 'list'>
limit : <class 'int'>
next : <class 'str'>
offset : <class 'int'>
previous : <class 'NoneType'>
total : <class 'int'>


In [6]:
print( f"Limit: {response['limit']}, Offset: {response['limit']}, Total: {response['limit']}, Length: {len(response['items'])}" )
print( response['next'] )

Limit: 100, Offset: 100, Total: 100, Length: 100
https://api.spotify.com/v1/playlists/6KI7A4MWrSM7EyKRUjxIi1/tracks?offset=100&limit=100


In [7]:
def get_playlist_tracks( playlist_id, access_token ):
    """ Get a list of track metadata for all playlist items """
    initURL      = BASE_URL + 'playlists/' + playlist_id + '/tracks/'
    response     = requests.get( initURL, headers = get_headers( access_token ) ).json()
    nextURL      = response['next']
    playlistItms = response['items'][:]
    while nextURL is not None:
        response = requests.get( nextURL, headers = get_headers( access_token ) ).json()
        nextURL  = response['next']
        playlistItms.extend( response['items'] )
    return playlistItms
    

# playlist_id  = '6KI7A4MWrSM7EyKRUjxIi1'


# print( f"Retrieved {len(playlistItms)} playlist items!" )
    




# Reorder Items

In [8]:
# FIXME, START HERE: https://developer.spotify.com/documentation/web-api/reference/reorder-or-replace-playlists-tracks
from math import ceil
study5_id  = "02iS5AFGp8YVuUUqcQf8ys"
s5Tracks   = get_playlist_tracks( study5_id, access_token )
Ntracks    = len(s5Tracks)
Nhalf      = int( ceil( Ntracks/2 ) )
print( f"Retrieved {len(s5Tracks)} playlist items!, Half is {Nhalf} tracks." )



Retrieved 331 playlist items!, Half is 166 tracks.


In [10]:
# NOTE: MOVE LIMIT IS 100, Need to break up big moves into multiple requests!

import json
requestURL = BASE_URL + 'playlists/' + study5_id + '/tracks/'
response   = requests.get( 
    requestURL, 
    headers = get_headers( access_token ),
    params  = { "data:" : {
        "range_start":   1,
        "insert_before": Ntracks+1,
        "range_length":  Nhalf,
    }, }
)
pprint( response )

<Response [200]>


In [11]:
response = response.json()
pprint( response )

{'href': 'https://api.spotify.com/v1/playlists/02iS5AFGp8YVuUUqcQf8ys/tracks?offset=0&limit=100',
 'items': [{'added_at': '2023-11-29T02:55:15Z',
            'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/31ytgsr7wdmiaroy77msqpiupdsi'},
                         'href': 'https://api.spotify.com/v1/users/31ytgsr7wdmiaroy77msqpiupdsi',
                         'id': '31ytgsr7wdmiaroy77msqpiupdsi',
                         'type': 'user',
                         'uri': 'spotify:user:31ytgsr7wdmiaroy77msqpiupdsi'},
            'is_local': False,
            'primary_color': None,
            'track': {'album': {'album_type': 'album',
                                'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2Vr1fub6eNIztdUNeFPvJj'},
                                             'href': 'https://api.spotify.com/v1/artists/2Vr1fub6eNIztdUNeFPvJj',
                                             'id': '2Vr1fub6eNIztdUNeFPvJj',
            